In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q peft~=0.14.0

# 訓練 PII 遮掩模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練 PII (個人識別資訊) 遮掩模型。我們將使用 `transformers` 套件中的 `SFTTrainer` ([Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)) 類別來訓練模型，。

In [2]:
# import garbage collector
import gc

import pandas as pd

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  TrainingArguments,
  Trainer,
)
from datasets import load_dataset, DatasetDict
from transformers import (
  DataCollatorForSeq2Seq,
  pipeline,
)

from typing import Any
from pydantic import BaseModel
from pprint import pprint

import torch

# 載入 PEFT 相關套件
from peft import LoraConfig, TaskType, PeftModel, get_peft_model
# 載入 SFTTrainer 相關套件
from trl import SFTConfig, SFTTrainer

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

/Users/jonas_cheng/.pyenv/versions/learning-transformer-with-huggingface-3.11.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 下載資料

In [90]:
# The full `train` split, only 50% of dataset
split = "train[:50%]" if device.type != 'mps' else "train[:1%]"
immutable_dataset = load_dataset("ai4privacy/pii-masking-65k", split=split)

### 資料包含什麼？

In [91]:
# Reserve 0.05% of the training set for testing
test_dataset = immutable_dataset.train_test_split(
  test_size=0.0005, # 0.05% of the data is used for testing
  shuffle=False, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=False, # Ensure that train and test sets are the same across runs
  )
immutable_dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })
# 顯示原始資料
immutable_dataset

DatasetDict({
    train: Dataset({
        features: ['masked_text', 'unmasked_text', 'token_entity_labels', 'tokenised_unmasked_text'],
        num_rows: 172
    })
    validation: Dataset({
        features: ['masked_text', 'unmasked_text', 'token_entity_labels', 'tokenised_unmasked_text'],
        num_rows: 43
    })
    test: Dataset({
        features: ['masked_text', 'unmasked_text', 'token_entity_labels', 'tokenised_unmasked_text'],
        num_rows: 1
    })
})

In [92]:
# 保留必要 features: 'masked_text', 'unmasked_text'
dataset = immutable_dataset.remove_columns(['token_entity_labels', 'tokenised_unmasked_text'])
# 顯示處理後的資料
dataset

DatasetDict({
    train: Dataset({
        features: ['masked_text', 'unmasked_text'],
        num_rows: 172
    })
    validation: Dataset({
        features: ['masked_text', 'unmasked_text'],
        num_rows: 43
    })
    test: Dataset({
        features: ['masked_text', 'unmasked_text'],
        num_rows: 1
    })
})

In [93]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset['train'].select(range(first_n_data)))

,masked_text,unmasked_text
0,"[PREFIX_1] [FIRSTNAME_1] [MIDDLENAME_1] [LASTNAME_1], as a [JOBDESCRIPTOR_1] [JOBTITLE_1] at [COMPANY_NAME_1], your knowledge of change management is vital for our company's transformation. We request you to create a change management strategy.","Mr. Adolphus Reagan Ziemann, as a Central Principal Applications Executive at McLaughlin, Nader and Purdy, your knowledge of change management is vital for our company's transformation. We request you to create a change management strategy."
1,"Hello [FIRSTNAME_1], would you please investigate the potential fallouts associated with the revisions in the [JOBAREA_1] department? Please incorporate your findings in your management strategy required previously.","Hello Hannah, would you please investigate the potential fallouts associated with the revisions in the Security department? Please incorporate your findings in your management strategy required previously."
2,We also request a review of our policies with respect to the upcoming changes and to bring in your expertise in case a policy change is advised. You can communicate the updates via email at [EMAIL_1].,We also request a review of our policies with respect to the upcoming changes and to bring in your expertise in case a policy change is advised. You can communicate the updates via email at Bartholome_Goldner85@yahoo.com.


## 訓練設定

In [7]:
# 訓練相關設定, 利用降低 batch size 提高 gradient accumulation steps 來節省記憶體
class Config(BaseModel):
  model_name: str = 'microsoft/Phi-3.5-mini-instruct'
  torch_dtype: Any = torch.bfloat16 # 半精度浮點數
  adam_epsilon: float = 1e-4 # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  saved_model_path: str = 'sample_data/saved_encoder_model' # path to save the trained model
  saved_lora_path: str = 'sample_data/saved_lora_model' # path to save the trained LORA model
  train_batch_size: int = 2 # size of the input batch in training
  eval_batch_size: int = 2 # size of the input batch in evaluation
  gradient_accumulation_steps: int = 2 # number of updates steps to accumulate before performing a backward/update pass
  epochs: int = 3 # number of times to iterate over the entire training dataset
  lr: float = 2e-5 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.

  # LORA 相關設定
  rank: int = 128 # rank of the PEFT model

config = Config(
  torch_dtype=torch.bfloat16 if device.type != 'mps' else torch.float16 # MPS 需要使用 torch.float16
)

## 先觀察 Fine-tuning 前的表現

### 載入 Tokenizer

* 如果沒有定義 `pad_token`，請定義一個 `pad_token`，並將其加入 Tokenizer 中。
* 如果 `padding_side` 不是 `right`，請將其設定為 `right`。

In [8]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
# 檢視 Tokenizer，是否存在 padding token 及 padding side 等資訊
pprint(tokenizer)

LlamaTokenizerFast(name_or_path='microsoft/Phi-3.5-mini-instruct', vocab_size=32000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special

In [9]:
# Add pad_token to the tokenizer
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  print('=== 設定 Padding Token ===')
  pprint(tokenizer)
# Make sure padding_side is 'right'
if tokenizer.padding_side != 'right':
  tokenizer.padding_side = 'right'
  print('=== 設定 Padding Side ===')
  pprint(tokenizer)

=== 設定 Padding Side ===
LlamaTokenizerFast(name_or_path='microsoft/Phi-3.5-mini-instruct', vocab_size=32000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, 

### 載入預訓練模型

由於 GPU 記憶體有限，我們將使用半精度進行模型 Fine-tuning。這邊需要留意，使用半精度進行 Fine-tuning 時，`TrainingArguments` 中的 `adam_epsilon` 需要設定為 `1e-4`。預設的 `adam_epsilon` 是 `1e-8`，這個值在半精度訓練時會出現問題。

In [10]:
# 半精度浮點數訓練
model = AutoModelForCausalLM.from_pretrained(
  config.model_name,
  torch_dtype=config.torch_dtype,
  low_cpu_mem_usage=True,
).to(device)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.48s/it]


In [11]:
# 獲取模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in model.named_parameters():
  print(f'{name}: {param.dtype}')

model.embed_tokens.weight: torch.float16
model.layers.0.self_attn.o_proj.weight: torch.float16
model.layers.0.self_attn.qkv_proj.weight: torch.float16
model.layers.0.mlp.gate_up_proj.weight: torch.float16
model.layers.0.mlp.down_proj.weight: torch.float16
model.layers.0.input_layernorm.weight: torch.float16
model.layers.0.post_attention_layernorm.weight: torch.float16
model.layers.1.self_attn.o_proj.weight: torch.float16
model.layers.1.self_attn.qkv_proj.weight: torch.float16
model.layers.1.mlp.gate_up_proj.weight: torch.float16
model.layers.1.mlp.down_proj.weight: torch.float16
model.layers.1.input_layernorm.weight: torch.float16
model.layers.1.post_attention_layernorm.weight: torch.float16
model.layers.2.self_attn.o_proj.weight: torch.float16
model.layers.2.self_attn.qkv_proj.weight: torch.float16
model.layers.2.mlp.gate_up_proj.weight: torch.float16
model.layers.2.mlp.down_proj.weight: torch.float16
model.layers.2.input_layernorm.weight: torch.float16
model.layers.2.post_attention_l

### 詠唱格式化 (Prompt Formatting)

先定義我們的詠唱 (Prompt) 格式。為此，我們將創建一個格式化函數。

In [17]:
system_message = 'Given the information below, mask the personal identifiable information.'

def instruction_formatter(x):
  text = f'''
    <|system|> {system_message}.
    <|user|> {x['unmasked_text']}
    <|assistant|>
  '''

  return text

### Fine-tuning 前的表現

In [13]:
# 載入預訓練模型
generator = pipeline(
  task='text-generation',
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [14]:
# 顯示預訓練模型預測結果
input = instruction_formatter(dataset['test'][0])
response = generator(
  input,
  max_new_tokens=128, # 限制最大生成字數
  repetition_penalty=1.5, # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)
)
print(response[0]['generated_text'])

/Users/jonas_cheng/.pyenv/versions/learning-transformer-with-huggingface-3.11.2/lib/python3.11/site-packages/transformers/pytorch_utils.py:325: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)



    <|system|> Given the information below, mask the personal identifiable information.
    <|user|> In order to finalize your registration for our Animal-assisted Therapy Program, please make the payment of Guarani 428.22. This can be made to our Auto Loan Account with account number 58114957. Once the payment is received, we will reach out to you with more information. Please use the provided contact (364) 467-8496 x139 for any queries related to your registration. Please use your GH5C2WDB7WVB30204 as a reference.
    <|assistant|>
   To complete participation in an animal assistance theraputic program: Kindly process remittance worth equivalent value [GUARANI_AMOUNT] using designated auto loan facility identified by unique identifier '[AUTOPLACEMENT]. Post transaction confirmation; communication shall ensue providing additional details pertinent thereof. For enquiries regarding said engagement utilise telecommunications channel labelled under code/reference tagged herein - ['REFERE

## 訓練模型

隨著 `trl` 的最新版本發布，現在支持流行的指令 (instruction) 和對話 (conversation) 數據集格式。這意味著我們只需要將數據集轉換為支持的格式之一，`trl` 會處理其餘的部分。這些格式包括：

* 指令格式 instruction format

```json
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

* 對話格式 conversational format

```json
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```


### 資料預處理

In [102]:
def create_conversation(dataset):
  rows = []
  unmasked_texts = dataset['unmasked_text']
  masked_texts = dataset['masked_text']
  for unmasked_text, masked_text in zip(unmasked_texts, masked_texts):
    rows.append([
        {"role": "system", "content": system_message},
        {"role": "user", "content": unmasked_text},
        {"role": "assistant", "content": masked_text}
      ],)
  return {'messages': rows}


In [110]:
conversation_dataset = dataset.map(
  create_conversation,
  batched=True,
  remove_columns=dataset['train'].column_names,
)

In [111]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(conversation_dataset['train'].select(range(first_n_data)))

,messages
0,"[{'content': 'Given the information below, mask the personal identifiable information.', 'role': 'system'}, {'content': 'Mr. Adolphus Reagan Ziemann, as a Central Principal Applications Executive at McLaughlin, Nader and Purdy, your knowledge of change management is vital for our company's transformation. We request you to create a change management strategy.', 'role': 'user'}, {'content': '[PREFIX_1] [FIRSTNAME_1] [MIDDLENAME_1] [LASTNAME_1], as a [JOBDESCRIPTOR_1] [JOBTITLE_1] at [COMPANY_NAME_1], your knowledge of change management is vital for our company's transformation. We request you to create a change management strategy.', 'role': 'assistant'}]"
1,"[{'content': 'Given the information below, mask the personal identifiable information.', 'role': 'system'}, {'content': 'Hello Hannah, would you please investigate the potential fallouts associated with the revisions in the Security department? Please incorporate your findings in your management strategy required previously.', 'role': 'user'}, {'content': 'Hello [FIRSTNAME_1], would you please investigate the potential fallouts associated with the revisions in the [JOBAREA_1] department? Please incorporate your findings in your management strategy required previously.', 'role': 'assistant'}]"
2,"[{'content': 'Given the information below, mask the personal identifiable information.', 'role': 'system'}, {'content': 'We also request a review of our policies with respect to the upcoming changes and to bring in your expertise in case a policy change is advised. You can communicate the updates via email at Bartholome_Goldner85@yahoo.com.', 'role': 'user'}, {'content': 'We also request a review of our policies with respect to the upcoming changes and to bring in your expertise in case a policy change is advised. You can communicate the updates via email at [EMAIL_1].', 'role': 'assistant'}]"


In [112]:
# 顯示單筆方便閱讀
pprint(conversation_dataset['train'][0])

{'messages': [{'content': 'Given the information below, mask the personal '
                          'identifiable information.',
               'role': 'system'},
              {'content': 'Mr. Adolphus Reagan Ziemann, as a Central Principal '
                          'Applications Executive at McLaughlin, Nader and '
                          'Purdy, your knowledge of change management is vital '
                          "for our company's transformation. We request you to "
                          'create a change management strategy.',
               'role': 'user'},
              {'content': '[PREFIX_1] [FIRSTNAME_1] [MIDDLENAME_1] '
                          '[LASTNAME_1], as a [JOBDESCRIPTOR_1] [JOBTITLE_1] '
                          'at [COMPANY_NAME_1], your knowledge of change '
                          "management is vital for our company's "
                          'transformation. We request you to create a change '
                          'management strategy.'

### LoRA 的訓練策略 - 降維打擊

LoRA（Low-Rank Adaptation）是一種用於訓練大型語言模型的技術，旨在提高訓練效率並減少計算資源的需求。以下是為何需要透過LoRA訓練的一些原因：

降低計算成本：LoRA 通過將模型的權重矩陣分解為低秩矩陣，顯著減少了參數的數量，從而降低了計算成本和內存需求。

加速訓練速度：由於參數數量減少，LoRA 可以加速模型的訓練過程，使得在相同的硬件資源下能夠更快地完成訓練。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png)

In [113]:
# 查看預訓練模型可訓練的參數量，其數量相當龐大，所以需要透過 Low Rank Approximation (LORA) 來降低參數量
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

Parameters: 3,821,079,552, Trainable Parameters: 3,821,079,552


#### PEFT 配置

`target_module`: 要降維的模型層，可以透過 `model.named_parameters` 查看。

In [114]:
# PEFT 配置
lora_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  r=config.rank,
  target_modules=['qkv_proj'],
)
pprint(lora_config)

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>,
           peft_type=<PeftType.LORA: 'LORA'>,
           auto_mapping=None,
           base_model_name_or_path=None,
           revision=None,
           inference_mode=False,
           r=128,
           target_modules={'qkv_proj'},
           exclude_modules=None,
           lora_alpha=8,
           lora_dropout=0.0,
           fan_in_fan_out=False,
           bias='none',
           use_rslora=False,
           modules_to_save=None,
           init_lora_weights=True,
           layers_to_transform=None,
           layers_pattern=None,
           rank_pattern={},
           alpha_pattern={},
           megatron_config=None,
           megatron_core='megatron.core',
           loftq_config={},
           eva_config=None,
           use_dora=False,
           layer_replication=None,
           runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False),
           lora_bias=False)


#### 取得 PEFT 模型

搭配預訓模型及 PEFT 配置，我們可以取得 PEFT 模型。我們可以觀察受到降維影響的模型層。

In [115]:
# 取得 PEFT 模型
peft_model = get_peft_model(
  model, # 預訓練模型
  lora_config, # PEFT 配置
)

/Users/jonas_cheng/.pyenv/versions/learning-transformer-with-huggingface-3.11.2/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [ ]:
# 取得 PEFT 模型, 觀察受 PEFT 影響的模型參數
peft_model

#### 調整 PEFT 模型精度

PEFT 模型的精度是 `torch.float32`，我們可以透過 `model.half()` 將其轉換為半精度。

In [ ]:
# 獲取 PERF 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

In [35]:
# 同樣採用半精度浮點數訓練
peft_model = peft_model.half()

In [116]:
# 查看可訓練的參數量
peft_model.print_trainable_parameters()

trainable params: 50,331,648 || all params: 3,871,411,200 || trainable%: 1.3001


### 定義訓練參數

In [117]:
training_args = SFTConfig(
  output_dir='sample_data/train_output_pii_masking',
  learning_rate=config.lr,
  per_device_train_batch_size=config.train_batch_size,
  per_device_eval_batch_size=config.eval_batch_size,
  gradient_accumulation_steps=config.gradient_accumulation_steps,
  num_train_epochs=config.epochs,
  weight_decay=config.weight_decay,
  eval_strategy='epoch', # 每個 epoch 評估一次
  save_strategy='epoch', # 每個 epoch 儲存一次
  load_best_model_at_end=True,
  report_to='none', # Disable wandb on colab
  adam_epsilon=config.adam_epsilon, # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  packing=True,
  max_seq_length=1024, # max sequence length for model and packing of the dataset
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    peft_config=lora_config,
    train_dataset=conversation_dataset['train'],
    eval_dataset=conversation_dataset['validation'],
)


Generating train split: 55 examples [00:00, 189.05 examples/s]
Generating train split: 14 examples [00:00, 510.03 examples/s]
/Users/jonas_cheng/.pyenv/versions/learning-transformer-with-huggingface-3.11.2/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


### 開始訓練

In [118]:
# 開始訓練，這可能需要一些時間
trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 18.09 GB, other allocations: 1.92 MB, max allowed: 18.13 GB). Tried to allocate 125.01 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
trainer.save_model()

#### 保存 LoRA 模型參數

In [40]:
# 保存 Lora 参数
peft_model.save_pretrained(config.saved_lora_path)

#### 合併 LoRA 模型參數

In [ ]:
# 合併原始模型和 Lora 参数
new_model = PeftModel.from_pretrained(model, config.saved_lora_path)

print("=== 合併前的模型結構 ===")
print(new_model)

In [ ]:
# 合併並卸載 Lora 参数
new_model.merge_and_unload()

print("=== 合併後的模型結構 ===")
print(new_model)

In [ ]:
# 保存合併後的模型
new_model.save_pretrained(config.saved_model_path)
tokenizer.save_pretrained(config.saved_model_path)

### 釋放資源

In [43]:
# 釋放 GPU 記憶體
del new_model
del trainer

peft_model.to('cpu')
del peft_model

model.to('cpu')
del model
torch.cuda.empty_cache()

gc.collect()

## 評估模型

### 載入微調後 Tokenizer

從已經完成訓練的模型取得 Tokenizer，可以留意這個訓練時保存下來的 Tokenizer 仍保有訓練時的設定，包涵 `pad_token` 和 `padding_side`。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
  config.saved_model_path
)
# 檢視 Tokenizer
pprint(tokenizer)

### 載入微調後模型

以半精度浮點數載入已經完成訓練的模型

In [ ]:
# 以半精度浮點數載入已經完成訓練的模型
model = AutoModelForCausalLM.from_pretrained(
  config.saved_model_path,
  low_cpu_mem_usage=True,
  torch_dtype=config.torch_dtype,
).to(device)

### Fine-tuning 後的表現

In [56]:
# 載入新模型
generator = pipeline(
  task='text-generation',
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 顯示新模型預測結果
input = instruction_formatter(dataset['test'][0])
response = generator(
  input,
  max_new_tokens=512, # 限制最大生成字數
  repetition_penalty=1.5, # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)
)
print(response[0]['generated_text'])